In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np


In [2]:
# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
# hyperparameters
num_epochs = 20
batch_size = 4
learning_rate = 0.001


In [4]:
# dataset has PILImage images of range [0,1]
# we transform them to tensor of normalized range [-1,1]

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])


In [5]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data',train=False, download=True, transform = transform)

train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [6]:
classes = ('plane','car','bird', 'cat','deer','dog','frog', 'horse','ship', 'truck')
classes

('plane',
 'car',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck')

In [7]:
# model architecture

class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1,16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x



In [8]:
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)
criterion.to(device)

CrossEntropyLoss()

In [9]:
n_total_steps = len(train_dataloader)
for epoch in range(num_epochs):
  for i, (images,labels) in enumerate(train_dataloader):
    # origin shape: [4,3,32,32] = 4,3,1024
    # input layer: 3 input channels, 6 output channels, 5 kernel size
    images = images.to(device)
    labels = labels.to(device)

    # forward pass

    outputs = model(images)
    loss = criterion(outputs,labels)


    # backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(i+1)% 2000==0:
      print(f'Epoch [{epoch+1}/{num_epochs}] Step [{i+1}/{n_total_steps}], loss: {loss.item():.4f}')

print('training finished')


Epoch [1/20] Step [2000/12500], loss: 2.3099
Epoch [1/20] Step [4000/12500], loss: 2.3283
Epoch [1/20] Step [6000/12500], loss: 2.2645
Epoch [1/20] Step [8000/12500], loss: 2.2905
Epoch [1/20] Step [10000/12500], loss: 2.1830
Epoch [1/20] Step [12000/12500], loss: 2.3782
Epoch [2/20] Step [2000/12500], loss: 1.9073
Epoch [2/20] Step [4000/12500], loss: 1.3551
Epoch [2/20] Step [6000/12500], loss: 1.3033
Epoch [2/20] Step [8000/12500], loss: 2.5156
Epoch [2/20] Step [10000/12500], loss: 2.0690
Epoch [2/20] Step [12000/12500], loss: 2.2087
Epoch [3/20] Step [2000/12500], loss: 1.5490
Epoch [3/20] Step [4000/12500], loss: 1.0035
Epoch [3/20] Step [6000/12500], loss: 1.3698
Epoch [3/20] Step [8000/12500], loss: 1.8513
Epoch [3/20] Step [10000/12500], loss: 1.9001
Epoch [3/20] Step [12000/12500], loss: 1.5299
Epoch [4/20] Step [2000/12500], loss: 1.2568
Epoch [4/20] Step [4000/12500], loss: 1.0385
Epoch [4/20] Step [6000/12500], loss: 2.1628
Epoch [4/20] Step [8000/12500], loss: 1.3900
Epoc

KeyboardInterrupt: 

In [10]:
with torch.no_grad():
  n_correct=0
  n_samples=0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_dataloader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    # max returns (value, index)
    _, predicted = torch.max(outputs,1)
    n_samples += labels.size(0)
    n_correct += (predicted ==labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if(label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] +=1

  acc = 100.0 * n_correct/n_samples
  print(f'Accuracy of the network : {acc:.4f}%')

  for i in range(10):
    acc= 100.0 * n_class_correct[i]/n_class_samples[i]
    print(f'Accuracy of class{classes[i]} : {acc:.4f} %')

Accuracy of the network : 56.4300%
Accuracy of classplane : 71.3000 %
Accuracy of classcar : 73.7000 %
Accuracy of classbird : 55.3000 %
Accuracy of classcat : 17.9000 %
Accuracy of classdeer : 35.1000 %
Accuracy of classdog : 52.6000 %
Accuracy of classfrog : 74.2000 %
Accuracy of classhorse : 72.1000 %
Accuracy of classship : 64.2000 %
Accuracy of classtruck : 47.9000 %
